In [ ]:
!pip install langchain
!pip install python-dotenv
!pip install google-generativeai
!pip install langchain-google-genai==2.0.7
!pip install langsmith
!pip install langchain_community
!pip install wikipedia
!pip install langchain-huggingface
!pip install faiss-cpu
!pip install arxiv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.31
    Uninstalling langchain-core-0.3.31:
      Successfully uninstalled langchain-core-0.3.31
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.15
    Uninstalling langchain-0.3.15:
      Successfully uninstalled langchain-0.3.15
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=cc791abf89cca2ad128ff2077a9520d590eb413f27a42daab0bfcfd29659c0bf
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

WikipediaAPIWrapper = WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=400)
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper)
wikipedia

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.11/dist-packages/wikipedia/__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=400))

In [ ]:
wikipedia.name

'wikipedia'

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from google.colab import userdata
import os

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGING_FACE_TOKEN')

In [ ]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
embeddings = HuggingFaceEmbeddings()
vectordb = FAISS.from_documents(documents,embeddings)
retriever = vectordb.as_retriever()
retriever

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x797e81679c10>, search_kwargs={})

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [ ]:
retriever_tool.name

'langsmith_search'

In [ ]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=300)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [ ]:
tools=[wikipedia,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.11/dist-packages/wikipedia/__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=400)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=2, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=300)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x797fa669cf40>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.fa

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGSMITH_KEY")
os.environ["LANGCHAIN_PROJECT"] = "langchain-1"

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)

In [ ]:
llm.predict("what all you know about amiba")

<ipython-input-16-0bba87d8ec78>:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("what all you know about amiba")


'Amoebas (also spelled amebas) are single-celled eukaryotic organisms belonging to the kingdom Protista.  Here\'s what we know about them:\n\n**Key Characteristics:**\n\n* **Single-celled:**  They are unicellular organisms, meaning their entire body is made up of just one cell.\n* **Eukaryotic:** Their cells contain a membrane-bound nucleus and other organelles.\n* **Amoeboid movement:** They move and feed using pseudopodia ("false feet"), temporary projections of their cytoplasm.  These extensions flow outwards, anchoring, and then pulling the rest of the cell along.\n* **Heterotrophic:** Most amoebas are heterotrophic, meaning they obtain their nutrition by consuming other organisms.  They engulf food particles through phagocytosis, a process where the cell membrane surrounds and engulfs the food, forming a food vacuole.\n* **Reproduction:** Amoebas typically reproduce asexually through binary fission, where the cell divides into two identical daughter cells.\n* **Variety of habitats

In [ ]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
from langchain.agents import create_openai_tools_agent

agent=create_openai_tools_agent(llm,tools,prompt)

In [ ]:
## Agent Executer
from langchain.agents import AgentExecutor

agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [ ]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith'}`


LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
LangSmith is a standalone platform that can be used on it's own no matter how you're creating your LLM applicatons.
In this tutorial, we'll walk you though logging your first trace in LangSmith using the LangSmith SDK and running an evaluation to measure the performance of your application. This example uses the OpenAI API, however you can use your provider of choice.
1. Install LangSmith‚Äã
PythonTypeScriptpip install -U langsmith openaiyarn add langsmith openai
2. Create an API key‚Äã
To create an API key head to the Settings page. Then click Create API Key.
3. Set up your environment‚Äã


{'input': 'Tell me about Langsmith',
 'output': "LangSmith is a platform for building production-grade LLM applications.  It allows you to monitor and evaluate your application, enabling faster and more confident deployments.  Key features include seamless integration with LangChain's open-source frameworks,  the ability to log traces, and running evaluations to measure performance.  You'll need an API key to use it.  The platform offers tools for observability, evaluation, prompt engineering, and deployment.\n"}

In [ ]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
allowed moves of arbitrary length. We show that the diameter of these graphs on
fibers of a fixed integeThe paper "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch studies graphs on lattice points.  The edges of these graphs are determined by a finite set of allowed moves of arbitrary length.  The paper focuses on the diameter of these graphs on fibers of a fixed integer.


> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch studies graphs on lattice points.  The edges of these graphs are determined by a finite set of allowed moves of arbitrary length.  The paper focuses on the diameter of these graphs on fibers of a fixed integer.\n'}